## Определение района Казани где расположена камера видеонаблюдения

In [1]:
import numpy as np
import pandas as pd
import re
import json
import folium # map rendering library
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim

In [2]:
df = pd.read_csv('lat-lon.csv', index_col=0)

In [3]:
with open('./kazan.geojson', "r") as outfile:
    kazan_geo = json.load(outfile)

Функция определения попадания в границы полигона

In [5]:
def isInPolygon(point, polygon) -> bool:
    isInside = False
    minLat = polygon[0][0]
    minLon = polygon[0][0]
    maxLat = polygon[0][1]
    maxLon = polygon[0][1]
    
    for poly in polygon:
        minLat = min(poly[0], minLat)
        maxLat = max(poly[0], maxLat)
        minLon = min(poly[1], minLon)
        maxLon = max(poly[1], maxLon)

    if (point[0] < minLat or point[0] > maxLat or point[1] < minLon or point[1] > maxLon):
        return False
    
    j = len(polygon) - 1
    for i in range(len(polygon)):
        if ( (polygon[i][1] > point[1]) != (polygon[j][1] > point[1]) and
            point[0] < (polygon[j][0] - polygon[i][0]) * (point[1] - polygon[i][1]) / (polygon[j][1] - polygon[i][1]) + polygon[i][0] ):
            isInside = not isInside
        j = i
    
    return isInside;

In [6]:
district_coordinates = {}

In [7]:
for district in kazan_geo['features']:
    district_coordinates[district['properties']['district']] = district['geometry']['coordinates'][0]

In [8]:
district_coordinates.keys()

dict_keys(['Московский район', 'Кировский район', 'Ново-Савиновский район', 'Советский район', 'Вахитовский район', 'Приволжский район', 'Авиастроительный район'])

In [9]:
district_coordinates['Авиастроительный район'] = district_coordinates['Авиастроительный район'][0] # cheat

In [10]:
def catchDistrict(row):
    #print(row)
    found_district = None
    for d in district_coordinates:
        if isInPolygon((row['Longitude'], row['Latitude']), district_coordinates[d]):
            found_district = d
            break
    return found_district

In [11]:
df['District'] = df.apply(catchDistrict, axis=1)

In [12]:
df

,Адрес установки камеры,ID папки,Latitude,Longitude,District
0,Альметьевск; Белоглазова 131; ТКО,0,54.902491,52.289705,None
1,Альметьевск; Белоглазова 151; ТКО,1,54.900211,52.287217,None
2,Альметьевск; Джалиля 15; ТКО,2,54.896967,52.296581,None
3,Альметьевск; Гафиатуллина 29Б; ТКО,3,54.905557,52.270927,None
4,Альметьевск; Гафиатуллина 39; ТКО,4,54.906408,52.266456,None
5,Альметьевск; Гафиатуллина 45; ТКО,5,54.905825,52.264366,None
6,Альметьевск; Гафиатуллина 47 (1): ТКО,6,54.906566,52.263848,None
7,Альметьевск; Гафиатуллина 47 (2); ТКО,7,54.906566,52.263848,None
8,Альметьевск; Гафиатуллина 51Б; ТКО,8,54.906745,52.262942,None
9,Альметьевск; Ленина 109; ТКО,9,54.898048,52.273717,None


In [13]:
df.to_csv('lat-lon.csv')